## Required libraries

In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import joblib
import pickle
import re
import snscrape.modules.twitter as sntwitter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score, confusion_matrix
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [181]:
real = pd.read_csv('/Users/lucashawranke/Documents/data-science/sarcasm-scanner/data/real_tweets.csv')
satire = pd.read_csv('/Users/lucashawranke/Documents/data-science/sarcasm-scanner/data/satire_tweets.csv')

In [125]:
# making lists to append all tweets/data to
# real = []
# satire = []

In [126]:
# using sntwitter to scrape tweets and append data to lists
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:cnnbrk').get_items()):
#     if i>5000:
#         break
#     real.append([tweet.content])

In [127]:
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:enews').get_items()):
#     if i>5000:
#         break
#     real.append([tweet.content])

In [128]:
# real = pd.DataFrame(real)

In [129]:
# real.to_csv('/Users/lucashawranke/Documents/data-science/sarcasm-scanner/data/real_tweets.csv') #mac
# real.to_csv('/Users/lucas/data-science/sarcasm-scanner/data/real_tweets.csv') #pc

In [131]:
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:theonion').get_items()):
#     if i>5000:
#         break
#     satire.append([tweet.content])

In [132]:
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:clickhole').get_items()):
#     if i>5000:
#         break
#     satire.append([tweet.content])

In [133]:
# satire = pd.DataFrame(satire)

In [134]:
# satire.to_csv('/Users/lucashawranke/documents/data-science/sarcasm-scanner/data/satire_tweets.csv') #pc

In [183]:
real['Authenticity'] = 'real'
satire['Authenticity'] = 'satire'
df = pd.concat([real, satire])

In [185]:
df.rename(columns={'0': 'Tweets'}, inplace=True)
df.rename(columns={'Unnamed: 0': 'No.'}, inplace=True)

In [186]:
df

,No.,Tweets,Authenticity
0,0,Nick Cannon is getting some rest after coming ...,real
1,1,Team USA eliminated from the World Cup after a...,real
2,2,A man has been arrested and charged with murde...,real
3,3,Right-wing conspiracy theorist Alex Jones file...,real
4,4,"America added a robust 263,000 jobs last month...",real
...,...,...,...
9997,9997,Are You A Widow? https://t.co/UWZLPewsKD https...,satire
9998,9998,Michael B. Jordan Said What?! https://t.co/y4F...,satire
9999,9999,Did ‘Sesame Street’ Go Too Far With Its Episod...,satire
10000,10000,Close Fucking Call: This Product That Got A Ne...,satire


In [191]:
df['Tweets'] = df['Tweets'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [192]:
df

,No.,Tweets,Authenticity
0,0,Nick Cannon is getting some rest after coming ...,real
1,1,Team USA eliminated from the World Cup after a...,real
2,2,A man has been arrested and charged with murde...,real
3,3,Right-wing conspiracy theorist Alex Jones file...,real
4,4,"America added a robust 263,000 jobs last month...",real
...,...,...,...
9997,9997,Are You A Widow?,satire
9998,9998,Michael B. Jordan Said What?!,satire
9999,9999,Did ‘Sesame Street’ Go Too Far With Its Episod...,satire
10000,10000,Close Fucking Call: This Product That Got A Ne...,satire


In [193]:
df.to_csv('/Users/lucashawranke/Documents/data-science/sarcasm-scanner/data/df.csv') #mac

### Creating/Running MultinomialNB Model

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=1)

In [195]:
X_train = real[:8000]
X_test = real[-2000:]

y_train = satire[:8000]
y_test = satire[-2000:]

In [190]:
mnb = MultinomialNB()
vectorizer = TfidfVectorizer(stop_words = 'english')

pipe = Pipeline([('vectorizer', vectorizer), ('nb', mnb)])
pipe.fit(train.Tweets, train.Authenticity)

predictions = pipe.predict(test.Tweets)
print('accuracy: ', accuracy_score(test['Authenticity'], predictions))
print('confusion matrix: ', confusion_matrix(test['Authenticity'], predictions))

accuracy:  0.9087728067983004
confusion matrix:  [[1925   91]
 [ 274 1711]]


### As you can see, this model has an accuracy of 90.8%. Although we might be able to do better, let's test it out and see what happens:

In [196]:
headline = np.array([["ISIS Takes Credit For Relatives Overstaying Welcome At Thanksgiving"]])
predictions = pipe.predict(headline[0])

print(predictions)

['satire']


### Correct! This article is from [a different](https://web.archive.org/web/20221204015813/http://www.cap-news.com/) satire news site. Let's save the model either way:

In [169]:
joblib.dump(pipe, '/Users/lucashawranke/Documents/data-science/sarcasm-scanner/mnb.pkl') #mac
# joblib.dump(pipe, '/Users/lucas/data-science/sarcasm-scanner/model.pkl') #pc

['/Users/lucashawranke/Documents/data-science/sarcasm-scanner/mnb.pkl']

In [ ]:
#mac
# with open('/Users/lucashawranke/Documents/data-science/sarcasm-scanner/model.pkl', 'wb') as files:
#     pickle.dump(pipe, files)
#pc
# with open('/Users/lucas/data-science/sarcasm-scanner/model.pkl', 'wb') as files:
#     pickle.dump(pipe, files)

### Creating LinearSVC Model

In [173]:
svc = LinearSVC()
vectorizer = TfidfVectorizer(stop_words = 'english')

pipe = Pipeline([('vectorizer', vectorizer), ('svc', svc)])
pipe.fit(train.Tweets, train.Authenticity)
predictions = pipe.predict(test.Tweets)
print('accuracy: ', accuracy_score(test['Authenticity'], predictions))
print('confusion matrix:', confusion_matrix(test['Authenticity'], predictions))

accuracy:  0.9492626843289178
confusion matrix: [[1890  126]
 [  77 1908]]


### Saving LinearSVC Model

In [172]:
joblib.dump(pipe, '/Users/lucashawranke/documents/data-science/sarcasm-scanner/lsvc.pkl') #mac
# joblib.dump(pipe, '/Users/lucas/data-science/sarcasm-scanner/model2.pkl') #pc

['/Users/lucashawranke/documents/data-science/sarcasm-scanner/lsvc.pkl']

In [157]:
lr = LogisticRegression()
vectorizer = TfidfVectorizer(stop_words = 'english')

pipe = Pipeline([('vectorizer', vectorizer), ('lr', lr)])
pipe.fit(train.Tweets, train.Authenticity)
predictions = pipe.predict(test.Tweets)
print('accuracy: ', accuracy_score(test['Authenticity'], predictions))
print('confusion matrix:', confusion_matrix(test['Authenticity'], predictions))

accuracy:  0.939265183704074
confusion matrix: [[1855  161]
 [  82 1903]]


In [158]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
vectorizer = TfidfVectorizer(stop_words = 'english')

pipe = Pipeline([('vectorizer', vectorizer), ('rf', rf)])
pipe.fit(train.Tweets, train.Authenticity)
predictions = pipe.predict(test.Tweets)
print('accuracy: ', accuracy_score(test['Authenticity'], predictions))
print('confusion matrix:', confusion_matrix(test['Authenticity'], predictions))

accuracy:  0.9232691827043239
confusion matrix: [[1822  194]
 [ 113 1872]]


In [160]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
vectorizer = TfidfVectorizer(stop_words = 'english')

pipe = Pipeline([('vectorizer', vectorizer), ('dt', dt)])
pipe.fit(train.Tweets, train.Authenticity)
predictions = pipe.predict(test.Tweets)
print('accuracy: ', accuracy_score(test['Authenticity'], predictions))
print('confusion matrix:', confusion_matrix(test['Authenticity'], predictions))

accuracy:  0.8470382404398901
confusion matrix: [[1575  441]
 [ 171 1814]]


In [161]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
vectorizer = TfidfVectorizer(stop_words = 'english')

pipe = Pipeline([('vectorizer', vectorizer), ('knn', knn)])
pipe.fit(train.Tweets, train.Authenticity)
predictions = pipe.predict(test.Tweets)
print('accuracy: ', accuracy_score(test['Authenticity'], predictions))
print('confusion matrix:', confusion_matrix(test['Authenticity'], predictions))

accuracy:  0.8780304923769058
confusion matrix: [[1843  173]
 [ 315 1670]]
